In [1]:
# Instructor Demo
%matplotlib inline
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json

# Google developer API key
from config import gkey

# Access maps with unique API key
gmaps.configure(api_key=gkey)

In [2]:
properties_path= "properties.csv"

In [3]:
# drop unnamed column tried all of these options
# properties_data.dropna(how='all', axis=1)
# properties_data=pd.read_csv('properties.csv', index_col=0)
properties_data = pd.read_csv(properties_path)
properties_data.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
properties_data.drop(["a"], axis=1, inplace=True)

properties_data.head()

,Address,AreaName,Price ($),lat,lng
0,"86 Waterford Dr Toronto, ON",Richview,999888,43.679882,-79.544266
1,"#80 - 100 BEDDOE DR Hamilton, ON",Chedoke Park B,399900,43.250000,-79.904396
2,"213 Bowman Street Hamilton, ON",Ainslie Wood East,479000,43.251690,-79.919357
3,"102 NEIL Avenue Hamilton, ON",Greenford,285900,43.227161,-79.767403
4,"#1409 - 230 King St Toronto, ON",Downtown,362000,43.651478,-79.368118


In [4]:
properties_data.dtypes

Address       object
AreaName      object
Price ($)      int64
lat          float64
lng          float64
dtype: object

In [5]:
# Dealing with special characters, removing each special character separtely
properties_data.columns = properties_data.columns.str.replace('$', ' ')
properties_data.columns = properties_data.columns.str.replace('(', ' ')
properties_data.columns = properties_data.columns.str.replace(')', ' ')
properties_data.head()

,Address,AreaName,Price,lat,lng
0,"86 Waterford Dr Toronto, ON",Richview,999888,43.679882,-79.544266
1,"#80 - 100 BEDDOE DR Hamilton, ON",Chedoke Park B,399900,43.250000,-79.904396
2,"213 Bowman Street Hamilton, ON",Ainslie Wood East,479000,43.251690,-79.919357
3,"102 NEIL Avenue Hamilton, ON",Greenford,285900,43.227161,-79.767403
4,"#1409 - 230 King St Toronto, ON",Downtown,362000,43.651478,-79.368118


In [6]:
properties_data.dtypes


Address       object
AreaName      object
Price          int64
lat          float64
lng          float64
dtype: object

In [7]:
# when running map function, there were found NaN data in lat and lng, use drop 
properties_drop=properties_data.dropna()

In [8]:
properties_drop.describe()


,Price,lat,lng
count,2.486800e+04,24868.000000,24868.000000
mean,5.669701e+05,43.939495,-79.382865
std,8.533163e+05,1.006421,2.383697
min,0.000000e+00,36.160065,-122.553154
25%,2.215000e+05,43.404707,-80.286264
50%,3.749000e+05,43.714592,-79.499923
75%,6.199000e+05,44.434961,-79.118414
max,3.250000e+07,53.851017,1.074519


In [9]:
#try filter by address, map is better with areaname
properties_tor=properties_drop[properties_drop['Address'].str.contains("Toronto")]
properties_tor.describe()

,Price,lat,lng
count,5.092000e+03,5092.000000,5092.000000
mean,7.499997e+05,43.709791,-79.392380
std,1.121114e+06,0.061736,0.110521
min,0.000000e+00,43.587688,-80.842804
25%,3.250000e+05,43.653177,-79.452715
50%,4.750000e+05,43.704050,-79.394135
75%,7.498880e+05,43.765161,-79.337746
max,2.680000e+07,44.232288,-76.504181


In [10]:
locations= properties_tor[["lat", "lng"]]
print(locations)

             lat        lng
0      43.679882 -79.544266
4      43.651478 -79.368118
5      43.686375 -79.328918
6      43.691193 -79.461662
11     43.664101 -79.439751
12     43.697842 -79.317368
13     43.635578 -79.441345
14     43.635578 -79.441345
15     43.666794 -79.388756
16     43.662872 -79.423233
17     43.749599 -79.270844
18     43.692314 -79.552635
19     43.642578 -79.376900
20     43.799480 -79.357445
21     43.788639 -79.207932
22     43.759003 -79.392075
23     43.799358 -79.377914
24     43.807091 -79.265533
25     43.682270 -79.443390
26     43.740097 -79.273987
27     43.733814 -79.250000
28     43.726189 -79.582047
29     43.774624 -79.298141
30     43.769413 -79.211456
31     43.684170 -79.516685
32     43.772118 -79.309311
33     43.805099 -79.211960
34     43.792187 -79.365669
35     43.650372 -79.382347
36     43.821087 -79.314796
...          ...        ...
22606  43.760998 -79.410202
22607  43.623608 -79.484077
22608  43.780670 -79.418343
22609  43.731674 -79

In [11]:
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

In [12]:
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations)
# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

In [13]:
#Retreive Postal Code and neighborhood
# Set up new column 
properties_tor['Postal Code'] = ""
properties_tor['Neighborhood'] = ""
properties_tor = properties_tor.reset_index(drop=True)
properties_tor.head()

/Users/ericH/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/ericH/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,Address,AreaName,Price,lat,lng,Postal Code,Neighborhood
0,"86 Waterford Dr Toronto, ON",Richview,999888,43.679882,-79.544266,,
1,"#1409 - 230 King St Toronto, ON",Downtown,362000,43.651478,-79.368118,,
2,"254A Monarch Park Ave Toronto, ON",Old East York,1488000,43.686375,-79.328918,,
3,"532 Caledonia Rd Toronto, ON",Fairbank,25,43.691193,-79.461662,,
4,"47 Armstrong Ave Toronto, ON",Wallace Emerson,113,43.664101,-79.439751,,


In [14]:
# Loop through the cities_pd and run a lat/long search for each city
for index, row in properties_tor.iterrows():
    

    latitude = row['lat']
    longitude = row['lng']
#Build the query url
    base_url = "https://maps.googleapis.com/maps/api/geocode/json?key=" + gkey + "&latlng=" + f"{latitude},{longitude}"



    # make request
    postalcode_neigh = requests.get(base_url)
    
    
    # convert to json
    postalcode_neigh = postalcode_neigh.json()

    properties_tor.loc[index, "Postal Code"] = postalcode_neigh["results"][0]["address_components"][-1]["long_name"]
    properties_tor.loc[index, "Neighborhood"] = postalcode_neigh["results"][0]["address_components"][2]["long_name"]

# Visualize to confirm lat lng appear
properties_tor.head()

IndexError: list index out of range

In [15]:
properties_tor

,Address,AreaName,Price,lat,lng,Postal Code,Neighborhood
0,"86 Waterford Dr Toronto, ON",Richview,999888,43.679882,-79.544266,M9R 2N8,Etobicoke
1,"#1409 - 230 King St Toronto, ON",Downtown,362000,43.651478,-79.368118,M5A 2P8,Old Toronto
2,"254A Monarch Park Ave Toronto, ON",Old East York,1488000,43.686375,-79.328918,M4J 4S5,East York
3,"532 Caledonia Rd Toronto, ON",Fairbank,25,43.691193,-79.461662,M6E 4V4,York
4,"47 Armstrong Ave Toronto, ON",Wallace Emerson,113,43.664101,-79.439751,M6H 1V9,Old Toronto
5,"3 Bracebridge Ave Toronto, ON",Old East York,599900,43.697842,-79.317368,M4C 2X6,East York
6,"#808 - 2167 Lake Shore Blvd Toronto, ON",Parkdale,97,43.635578,-79.441345,M6K 3C1,Old Toronto
7,"#2706 - 2167 Lake Shore Blvd Toronto, ON",Parkdale,49,43.635578,-79.441345,M6K 3C1,Old Toronto
8,"#710 - 1080 Bay St Toronto, ON",Downtown,805900,43.666794,-79.388756,M5S 0A5,Old Toronto
9,"#A - 753 Shaw St Toronto, ON",Midtown Toronto,97,43.662872,-79.423233,M6G 3L9,Old Toronto
